In [5]:
# import os
import os.path as op
from pathlib import Path
import numpy as np
from collections import defaultdict
# import matplotlib
from matplotlib import pyplot as plt
from matplotlib.offsetbox import AnchoredText
import mne

from scipy import stats
from mne.stats import permutation_cluster_1samp_test

from library import helpers, config


In [3]:

def get_evokeds_dict(epo_part, sub_list,  plt_dict):
    evokeds = defaultdict(list)
    for sub in sub_list:  
        subID = 'VME_S%02d' % sub
        fname = op.join(config.path_epos_sorted, epo_part,
                        'difference', subID + '-epo.fif')
        epos = mne.read_epochs(fname, verbose=False)
        epos.pick_channels(config.chans_CDA_dict['Left'])
        epos.crop(plt_dict[epo_part]['xmin'], plt_dict[epo_part]['xmax'])
        event_dict = helpers.get_event_dict(epos.event_id)
        for load in ['LoadLow', 'LoadHigh']:
            evoked_load = epos[event_dict[load]].copy().average()   
            evokeds[load].append(evoked_load)
            for ecc in ['EccS', 'EccM', 'EccL']:
                if load == 'LoadLow':  # we don't want to do this twice
                    evoked_ecc = epos[event_dict[ecc]].copy().average()  
                    evokeds[ecc].append(evoked_ecc)
                # Interaction:
                evoked_interac = epos[event_dict[load]][event_dict[ecc]].copy()
                evoked_interac = evoked_interac.average()
                evokeds[load+ecc].append(evoked_interac)
        evokeds['all'].append(epos.copy().average())
    return(evokeds)


def plot_main_cda(ax, evokeds):
    # Plot main curve CDA:
    label_dict = dict()
    label_dict['CDA: Contra - Ipsi'] = evokeds['all']

    res = mne.viz.plot_compare_evokeds(label_dict, 
                                combine='mean', 
                                #colors = {k: config.colors[k] for k in plt_dict.keys()},
                                vlines=[0, 0.2, 2.2],
                                ci=True,
                                ylim=dict(eeg=[-1.5,1.5]),
                                title=None, 
                                axes = ax, 
                                show = False, 
                                truncate_yaxis=False, 
                                truncate_xaxis=False
                                )

    ax.legend(loc=1, prop={'size': 9})
    ax.axvspan(0, 0.2, color='grey', alpha=0.3)
    ax.axvspan(2.2, 2.3, color='grey', alpha=0.3)
    ax.set_aspect(0.25)
    ax.set_title('')
    ax.xaxis.label.set_size(9)
    ax.yaxis.label.set_size(9)
    plt.xticks(fontsize=9)
    #ax.legend(prop=dict(size=9))



In [6]:

# define dict used for plotting of different epo parts:
plt_dict = defaultdict(dict)
pp = {'t_stimon':  0, 
      'xmin': -0.2, 
      'xmax': 2.3}
plt_dict['stimon'] = pp


In [7]:

sub_list = np.setdiff1d(np.arange(1, 28), config.ids_missing_subjects +
                        config.ids_excluded_subjects)

epo_part = 'stimon'  #in ['stimon']:

# get the evokeds: 
evokeds = get_evokeds_dict(epo_part, sub_list, plt_dict)

# Plot main CDA wave:
plt.subplots(1,1,figsize=(12,4))
ax = plt.axes()
plot_main_cda(ax, evokeds)

FileNotFoundError: [Errno 2] No such file or directory: '/cobra/ptmp/fklotzsche/Experiments/vMemEcc/vMemEcc/Data/DataMNE/EEG/07_epos_sorted/stimon/difference/VME_S01-epo.fif'